# Demo of thesis "Outdoor localization with Wi-Fi"

Author: Darina Dementyeva

In [10]:
import numpy as np
import math
from scipy import stats

## Input data

In [1]:
#room size
n = 100
m = 100

In [6]:
#define params of signal propagation distribution

mu = np.zeros((n, m)) # mean
alpha = 3
d0 = 4
p_d0 = -53
for i in range(0, n):
    for j in range(0, m): 
        d = math.sqrt((49 - i)*(49 - i) + (49 - j)*(49 - j)) + 4
        mu[i][j] = p_d0 - 10 * alpha * math.log10(d / d0)

In [7]:
sigma = np.ones((n, m)) #variance

In [106]:
#start point
x0 = 0
y0 = 0

In [105]:
#start velocity
v0 = 0

In [60]:
#time interval
dt = 1

## Positioning Algorithm

Some model parameters:

In [14]:
#person's motion: acceleration distribution
mu_a = 0
sigma_a = 1

In [15]:
#number of samples
K = 100

Supplementary functions:

In [63]:
def cond_prob(rssi, n, m, mu, sigma):
    cond_prob = np.zeros((n, m))
    for i in range(0, n):
        for j in range(0, m):
            cond_prob[i][j] = stats.norm(mu[i][j], sigma[i][j]).pdf(rssi)
            
    return cond_prob

In [97]:
def sample_generation(mu_a, sigma_a, K):
    a = np.random.normal(mu_a, sigma_a, K) #can be changed
    p_a = []
    for i in range(0, K):
        p_a.append(stats.norm(mu_a, sigma_a).pdf(a[i])) #can be changed
        
    return [a, p_a]

In [119]:
def motion_map(x, y, v, mu_a, sigma_a, K):
    samples  = sample_generation(mu_a, sigma_a, K)
    motion_prob = np.zeros((n, m))
    acceleration = np.zeros((n, m))
    for i in range(0, len(samples[0])):
        a = samples[0][i]
        p_a = samples[1][i]
        x_t = int(round(x + v*dt + a*a*dt/2, 0))
        y_t = int(round(y + v*dt + a*a*dt/2, 0))
        if x_t >= n | y_t >=m: #deal with going beyond the boundaries 
            x_t = x
            y_t = y
            a = 0
        motion_prob[x_t][y_t] = p_a
        acceleration[x_t][y_t] = a
        
    return motion_prob, acceleration

Path calculation: for input we take the array of rssi measurements

In [124]:
def path_estimation(RSSI, x, y, v):
    path_est = []
    path_est.append((0,0))
    #print(str(0) + " " + str(0))
    for rssi in RSSI:
        rssi_prob = cond_prob(rssi, n, m, mu, sigma)
        motion_prob, acceleration = motion_map(x, y, v, mu_a, sigma_a, K)
        prob = np.multiply(rssi_prob, motion_prob)
        best_samples = np.where(prob==prob.max()) #choose not just bust and first but adequate
        x = best_samples[0][0]
        y = best_samples[1][0]
        print(best_samples)
        print(str(x) + " " + str(y))
        a = acceleration[x][y]
        
        path_est.append((x, y))
        v = v + a*dt
        
    return path_est

## Test generation

In [123]:
def path_generation():
    length = np.random.randint(5, 10)

    path = [(0,0)]
    rssi = []
    rssi.append(np.random.normal(mu[0][0], sigma[0][0], 1))

    for i in range(1, length):
        dx = np.random.randint(0, 3) #change to generation with acceleration
        dy = np.random.randint(0, 3)
        x = path[-1][0] + dx
        y = path[-1][1] + dy
        #print(str(dx) + " " + str(dy))
        path.append((x,y))
        rssi.append(np.random.normal(mu[x][y], sigma[x][y], 1))
        
    return rssi, path

In [50]:
N = 5
paths = []

for i in range(0,5):
    paths.append(path_generation())

## Model evaluation

In [132]:
def error(path, path_est):
    error = 0
    for i in range(0, len(path)):
        error = error + (path[i][0] - path_est[i][0])**2 + (path[i][1] - path_est[i][1])**2
    error = math.sqrt(error)

    print("Error : " + str(error))

In [92]:
RSSI, path = path_generation()
path

[(0, 0), (0, 1), (1, 1), (1, 3), (2, 5)]

In [125]:
path_est = path_estimation(RSSI, x0, y0, v0)

(array([0]), array([0]))
0 0
(array([0]), array([0]))
0 0
(array([1]), array([1]))
1 1
(array([3]), array([3]))
3 3
(array([4]), array([4]))
4 4


In [126]:
path_est

[(0, 0), (0, 0), (0, 0), (1, 1), (3, 3), (4, 4)]

In [133]:
error(path, path_est)

Error : 3.4641016151377544
